# **Date Calculator | Days and Business Days**

In [1]:
# importing all the modules
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [2]:
# printing the current version of the selenium
print(webdriver.__version__)

4.18.1


In [3]:
# creating the chrome instance
service = Service(executable_path="./chromedriver.exe")
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service = service, options = options)

In [4]:
# opening the web page
driver.get("https://www.calculatorsoup.com/calculators/time/date-day.php")
driver.maximize_window()

In [5]:
from selenium.webdriver.support.ui import Select
date_format = Select(driver.find_element('id', 'given_format'))
date_format.select_by_visible_text("mm/dd/yyyy")  # text visible to the user

In [6]:
import pandas as pd
date_df = pd.read_excel("./BusinessDayCalculation.xlsx", sheet_name="Sheet1")
date_df.head()

,Serial_No,Start_Date,Operator,No_of_Days,End_Date,End_Date_BD,End_Date_BD_SI
0,1,2021-10-20,-,55,2021-08-26,2021-08-04,2021-08-17
1,2,2014-01-20,+,74,2014-04-04,2014-05-02,2014-04-16
2,3,2021-04-06,+,88,2021-07-03,2021-08-06,2021-07-17
3,4,2018-07-19,-,35,2018-06-14,2018-05-31,2018-06-08
4,5,2016-10-02,+,58,2016-11-29,2016-12-21,2016-12-08


In [7]:
date_df.insert(column="Status_1", value=None, loc=5)
date_df.insert(column="Status_2", value=None, loc=7)
date_df.insert(column="Status_3", value=None, loc=9)
date_df.head()

,Serial_No,Start_Date,Operator,No_of_Days,End_Date,Status_1,End_Date_BD,Status_2,End_Date_BD_SI,Status_3
0,1,2021-10-20,-,55,2021-08-26,None,2021-08-04,None,2021-08-17,None
1,2,2014-01-20,+,74,2014-04-04,None,2014-05-02,None,2014-04-16,None
2,3,2021-04-06,+,88,2021-07-03,None,2021-08-06,None,2021-07-17,None
3,4,2018-07-19,-,35,2018-06-14,None,2018-05-31,None,2018-06-08,None
4,5,2016-10-02,+,58,2016-11-29,None,2016-12-21,None,2016-12-08,None


In [8]:
current_row = date_df.iloc[0]
start_date = str(current_row.Start_Date)
print (start_date)
year_value = start_date.split("-")[0]
month_value = start_date.split("-")[1]
date_value = start_date.split("-")[2].split(" ")[0]
print (f"{year_value}, {month_value}, {date_value}")

2021-10-20 00:00:00
2021, 10, 20


In [9]:
# updating Status_1
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime

for i in range(date_df.shape[0]):
# for i in range(1):
    current_row = date_df.iloc[i]
    date_tf = driver.find_element("id", "dateInputD")
    month_tf = driver.find_element("id", "dateInputM")
    year_tf = driver.find_element("id", "dateInputY")
    operator_value = Select(driver.find_element('id', 'operator'))
    days_tf = driver.find_element("id", "days")
    calculateButton = driver.find_element("id", "calculateButton")
    
    start_date = str(current_row.Start_Date)
    year_value = start_date.split("-")[0]
    month_value = start_date.split("-")[1]
    date_value = start_date.split("-")[2].split(" ")[0]

    year_tf.clear(); year_tf.send_keys(year_value)
    month_tf.clear(); month_tf.send_keys(month_value)
    date_tf.clear(); date_tf.send_keys(date_value)
    # print (current_row.Operator, type(current_row.Operator))
    if (current_row.Operator == "+"): operator_value.select_by_value("add")
    else: operator_value.select_by_value("subtract")
        
    days_tf.clear(); days_tf.send_keys(str(current_row.No_of_Days))
    calculateButton.click()
    answerlist = driver.find_element(By.XPATH, "//div[@id='answer']")
    actual_date = datetime.strptime(answerlist.text.split("\n")[3], "%m/%d/%Y").date()
    expected_date = datetime.strptime(current_row.End_Date.strftime("%m/%d/%Y"), "%m/%d/%Y").date()
    # print (actual_date, type(actual_date), expected_date, type(expected_date))
    # if (actual_date == expected_date): current_row.Status_1 = "PASSED"
    # else: current_row.Status_1 = "FAILED"
    # date_df.iloc[i] = current_row
    if (actual_date == expected_date): date_df.loc[date_df.Serial_No == (i + 1), 'Status_1'] = "PASSED"
    else: date_df.loc[date_df.Serial_No == (i + 1), 'Status_1'] = "FAILED"

In [10]:
date_df

,Serial_No,Start_Date,Operator,No_of_Days,End_Date,Status_1,End_Date_BD,Status_2,End_Date_BD_SI,Status_3
0,1,2021-10-20,-,55,2021-08-26,PASSED,2021-08-04,None,2021-08-17,None
1,2,2014-01-20,+,74,2014-04-04,PASSED,2014-05-02,None,2014-04-16,None
2,3,2021-04-06,+,88,2021-07-03,PASSED,2021-08-06,None,2021-07-17,None
3,4,2018-07-19,-,35,2018-06-14,PASSED,2018-05-31,None,2018-06-08,None
4,5,2016-10-02,+,58,2016-11-29,PASSED,2016-12-21,None,2016-12-08,None
5,6,2023-06-29,-,39,2023-05-21,PASSED,2023-05-05,None,2023-05-15,None
6,7,2021-11-05,+,32,2021-12-07,PASSED,2021-12-21,None,2021-12-13,None
7,8,2008-06-17,-,85,2008-03-24,PASSED,2008-02-19,None,2008-03-10,None
8,9,2018-02-18,-,89,2017-11-21,PASSED,2017-10-17,None,2017-11-07,None
9,10,2022-04-04,+,44,2022-05-18,PASSED,2022-06-03,None,2022-05-25,None


In [11]:
# updating Status_2
for i in range(date_df.shape[0]):
# for i in range(1):
    date_tf = driver.find_element("id", "dateInputD")
    month_tf = driver.find_element("id", "dateInputM")
    year_tf = driver.find_element("id", "dateInputY")
    operator_value = Select(driver.find_element('id', 'operator'))
    days_tf = driver.find_element("id", "days")
    calculateButton = driver.find_element("id", "calculateButton")
    checkbox1 = driver.find_element("id", "business")
    if (not checkbox1.is_selected()): checkbox1.click()
    
    current_row = date_df.iloc[i]
    # print (current_row)
    start_date = str(current_row.Start_Date)
    year_value = start_date.split("-")[0]
    month_value = start_date.split("-")[1]
    date_value = start_date.split("-")[2].split(" ")[0]
    
    year_tf.clear(); year_tf.send_keys(year_value)
    month_tf.clear(); month_tf.send_keys(month_value)
    date_tf.clear(); date_tf.send_keys(date_value)
    # print (current_row.Operator, type(current_row.Operator))
    if (current_row.Operator == "+"): operator_value.select_by_value("add")
    else: operator_value.select_by_value("subtract")
        
    days_tf.clear(); days_tf.send_keys(str(current_row.No_of_Days))
    calculateButton.click()
    answerlist = driver.find_element(By.XPATH, "//div[@id='answer']")
    actual_date = datetime.strptime(answerlist.text.split("\n")[3], "%m/%d/%Y").date()
    expected_date = datetime.strptime(current_row.End_Date_BD.strftime("%m/%d/%Y"), "%m/%d/%Y").date()
    # print(actual_date, expected_date, actual_date == expected_date)
    if (actual_date == expected_date): date_df.loc[date_df.Serial_No == (i + 1), 'Status_2'] = "PASSED"
    else: date_df.loc[date_df.Serial_No == (i + 1), 'Status_2'] = "FAILED"

In [12]:
date_df

,Serial_No,Start_Date,Operator,No_of_Days,End_Date,Status_1,End_Date_BD,Status_2,End_Date_BD_SI,Status_3
0,1,2021-10-20,-,55,2021-08-26,PASSED,2021-08-04,PASSED,2021-08-17,None
1,2,2014-01-20,+,74,2014-04-04,PASSED,2014-05-02,PASSED,2014-04-16,None
2,3,2021-04-06,+,88,2021-07-03,PASSED,2021-08-06,PASSED,2021-07-17,None
3,4,2018-07-19,-,35,2018-06-14,PASSED,2018-05-31,PASSED,2018-06-08,None
4,5,2016-10-02,+,58,2016-11-29,PASSED,2016-12-21,PASSED,2016-12-08,None
5,6,2023-06-29,-,39,2023-05-21,PASSED,2023-05-05,PASSED,2023-05-15,None
6,7,2021-11-05,+,32,2021-12-07,PASSED,2021-12-21,PASSED,2021-12-13,None
7,8,2008-06-17,-,85,2008-03-24,PASSED,2008-02-19,PASSED,2008-03-10,None
8,9,2018-02-18,-,89,2017-11-21,PASSED,2017-10-17,PASSED,2017-11-07,None
9,10,2022-04-04,+,44,2022-05-18,PASSED,2022-06-03,PASSED,2022-05-25,None


In [13]:
# updating Status_3
for i in range(date_df.shape[0]):
# for i in range(1):
    date_tf = driver.find_element("id", "dateInputD")
    month_tf = driver.find_element("id", "dateInputM")
    year_tf = driver.find_element("id", "dateInputY")
    operator_value = Select(driver.find_element('id', 'operator'))
    days_tf = driver.find_element("id", "days")
    calculateButton = driver.find_element("id", "calculateButton")
    checkbox1 = driver.find_element("id", "business")
    checkbox2 = driver.find_element("id", "business_days_6")
    if (not checkbox1.is_selected()): checkbox1.click()
    if (not checkbox2.is_selected()): checkbox2.click()
    
    current_row = date_df.iloc[i]
    # print (current_row)
    start_date = str(current_row.Start_Date)
    year_value = start_date.split("-")[0]
    month_value = start_date.split("-")[1]
    date_value = start_date.split("-")[2].split(" ")[0]
    
    year_tf.clear(); year_tf.send_keys(year_value)
    month_tf.clear(); month_tf.send_keys(month_value)
    date_tf.clear(); date_tf.send_keys(date_value)
    # print (current_row.Operator, type(current_row.Operator))
    if (current_row.Operator == "+"): operator_value.select_by_value("add")
    else: operator_value.select_by_value("subtract")
        
    days_tf.clear(); days_tf.send_keys(str(current_row.No_of_Days))
    calculateButton.click()
    answerlist = driver.find_element(By.XPATH, "//div[@id='answer']")
    actual_date = datetime.strptime(answerlist.text.split("\n")[3], "%m/%d/%Y").date()
    expected_date = datetime.strptime(current_row.End_Date_BD_SI.strftime("%m/%d/%Y"), "%m/%d/%Y").date()
    # print(actual_date, expected_date, actual_date == expected_date)
    if (actual_date == expected_date): date_df.loc[date_df.Serial_No == (i + 1), 'Status_3'] = "PASSED"
    else: date_df.loc[date_df.Serial_No == (i + 1), 'Status_3'] = "FAILED"

In [14]:
date_df

,Serial_No,Start_Date,Operator,No_of_Days,End_Date,Status_1,End_Date_BD,Status_2,End_Date_BD_SI,Status_3
0,1,2021-10-20,-,55,2021-08-26,PASSED,2021-08-04,PASSED,2021-08-17,PASSED
1,2,2014-01-20,+,74,2014-04-04,PASSED,2014-05-02,PASSED,2014-04-16,PASSED
2,3,2021-04-06,+,88,2021-07-03,PASSED,2021-08-06,PASSED,2021-07-17,PASSED
3,4,2018-07-19,-,35,2018-06-14,PASSED,2018-05-31,PASSED,2018-06-08,PASSED
4,5,2016-10-02,+,58,2016-11-29,PASSED,2016-12-21,PASSED,2016-12-08,PASSED
5,6,2023-06-29,-,39,2023-05-21,PASSED,2023-05-05,PASSED,2023-05-15,PASSED
6,7,2021-11-05,+,32,2021-12-07,PASSED,2021-12-21,PASSED,2021-12-13,PASSED
7,8,2008-06-17,-,85,2008-03-24,PASSED,2008-02-19,PASSED,2008-03-10,PASSED
8,9,2018-02-18,-,89,2017-11-21,PASSED,2017-10-17,PASSED,2017-11-07,PASSED
9,10,2022-04-04,+,44,2022-05-18,PASSED,2022-06-03,PASSED,2022-05-25,PASSED


In [15]:
# Writing back the DataFrame content to one Excel Workbook
date_df.to_excel("./BusinessDayCalculation-1.xlsx", sheet_name="Sheet1")

In [16]:
driver.quit()